In [1]:
import pandas as pd
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.optimize import curve_fit
from astropy.timeseries import LombScargle
from scipy.signal import find_peaks
import os

In [2]:
archivo = fits.open('TCampo_b279.fits')
archivo1 = fits.open('PVariablesb279.fits')
HJD=archivo[1].data
Error=archivo1[4].data
Ks=archivo1[3].data
archivo1.info()

Filename: PVariablesb279.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  KS_ALE        1 ImageHDU         7   (1280954,)   float64   
  2  EKS_ALE       1 ImageHDU         8   (68, 100000)   float64   
  3  KS_PV         1 ImageHDU         8   (68, 181)   float64   
  4  EKS_PV        1 ImageHDU         8   (68, 181)   float64   


In [3]:
def eliminar_nan(error, ks, HJD):
    indices_validos = ~np.isnan(ks)
    error_filtrado = [error[i] for i in range(len(ks)) if indices_validos[i]]
    ks_filtrado = [ks[i] for i in range(len(ks)) if indices_validos[i]]
    HJD_filtrado = [HJD[i] for i in range(len(ks)) if indices_validos[i]]
    return error_filtrado, ks_filtrado, np.round(HJD_filtrado,8)

ks_nan=[]
error_nan=[]
HJD_na=[]
y=0
while y<len(Ks):
    error_filtrado, ks_filtrado, HJD_filtrado = eliminar_nan(Error[y], Ks[y], HJD-2400000)
    ks_nan.append(ks_filtrado)
    error_nan.append(error_filtrado)
    HJD_na.append(HJD_filtrado)
    y+=1

# Variables Periodicas y LPV

In [4]:
# Abre el archivo de texto en modo lectura
with open('B279.txt', 'r') as file:
    # Lee cada línea del archivo
    lines = file.readlines()

# Inicializa una lista vacía para almacenar los elementos de la primera columna
posibles_variables = []

# Itera sobre cada línea del archivo
for line in lines:
    # Divide la línea en columnas separadas por un espacio en blanco
    columnas = line.split()
    # Añade el primer elemento (primera columna) a la lista primera_columna
    posibles_variables.append(columnas[0])
    
# Abre el archivo de texto en modo lectura
with open('var_per', 'r') as file:
    # Lee cada línea del archivo
    lines = file.readlines()

# Inicializa una lista vacía para almacenar los elementos de la primera columna
periodicas_variables = []
periodos_periodicas = []
# Itera sobre cada línea del archivo
for line in lines:
    # Divide la línea en columnas separadas por un espacio en blanco
    columnas = line.split()
    # Añade el primer elemento (primera columna) a la lista primera_columna
    periodicas_variables.append(columnas[0])
    periodos_periodicas.append(float(columnas[1]))
    

# Extraer los números después de "Datosb279" usando list comprehension
posiciones_periodicas = [int(item.split('Datosb279')[1].split('.')[0]) for item in periodicas_variables]


# Abre el archivo de texto en modo lectura
with open('var_long', 'r') as file:
    # Lee cada línea del archivo
    lines = file.readlines()

# Inicializa una lista vacía para almacenar los elementos de la primera columna
LPV = []
periodos_LPV = []
# Itera sobre cada línea del archivo
for line in lines:
    # Divide la línea en columnas separadas por un espacio en blanco
    columnas = line.split()
    # Añade el primer elemento (primera columna) a la lista primera_columna
    LPV.append(columnas[0])
    periodos_LPV.append(float(columnas[1]))
    

# Extraer los números después de "Datosb279" usando list comprehension
posiciones_LPV = [int(item.split('Datosb279')[1].split('.')[0]) for item in LPV]

In [8]:
Series_periodicas=[]
HJD_periodicas=[]
error_periodicas=[]
Series_LPV=[]
HJD_LPV=[]
error_LPV=[]
for k in posiciones_periodicas: 
    Series_periodicas.append(ks_nan[k])
    HJD_periodicas.append(HJD_na[k])
    error_periodicas.append(error_nan[k])
for l in posiciones_LPV: 
    Series_LPV.append(ks_nan[l])
    HJD_LPV.append(HJD_na[l])
    error_LPV.append(error_nan[l])
x=0
while x < len(Series_periodicas): 
    Sp=Series_periodicas[x]
    Pp=periodos_periodicas[x]
    Datep=HJD_periodicas[x]
    t_0p = Datep[0]
    name= posiciones_periodicas[x]
    # 3. Calcular la fase para cada punto de tus datos
    fasep = ((Datep - t_0p) / Pp) - np.floor((Datep - t_0p) / Pp)
    plt.scatter(fasep, Sp, s=10, c='blue', alpha=0.5)
    plt.xlabel('Phase')
    plt.ylabel('Ks')
    # Invertir el eje y
    plt.gca().invert_yaxis()
    plt.title('Light curve b2791'+ str(name))
    # Muestra la leyenda con el período promedio
    plt.legend(['Period= {:.2f} days'.format(Pp)])
    plt.scatter(fasep + 1, Sp, s=10, c='blue', alpha=0.5)
    plt.scatter(fasep + 2, Sp, s=10, c='blue', alpha=0.5)
    # Guarda la gráfica en un archivo
    plt.savefig('Light curve b2791'+str(name)+'.png')
    plt.close()  # Cerrar la figura para evitar superposiciones
    #plt.show()
    x+=1
y=0
while y< len(Series_LPV):
    S=Series_LPV[y]
    P=periodos_LPV[y]
    Date=HJD_LPV[y]
    t_0 = Date[0]
    name= posiciones_LPV[y]
    # 3. Calcular la fase para cada punto de tus datos
    fase = ((Date - t_0) / P) - np.floor((Date - t_0) / P)
    plt.scatter(fase, S, s=10, c='blue', alpha=0.5)
    plt.xlabel('Phase')
    plt.ylabel('Ks')
    # Invertir el eje y
    plt.gca().invert_yaxis()
    plt.title('Light curve b2791'+ str(name))
    # Muestra la leyenda con el período promedio
    plt.legend(['Period= {:.2f} days'.format(P)])
    plt.scatter(fase + 1, S, s=10, c='blue', alpha=0.5)
    # Guarda la gráfica en un archivo
    plt.savefig('Light curve b2791'+str(name)+'.png')
    plt.close()  # Cerrar la figura para evitar superposiciones
    #plt.show()
    y+=1
print("Save")

Save


In [10]:
# Función para llenar las listas con NaNs para igualar la longitud

def llenar_con_nan(lista, longitud_maxima):
    return [np.pad(sublista, (0, longitud_maxima - len(sublista)), mode='constant', constant_values=np.nan) for sublista in lista]

# Obtener la longitud máxima de todas las listas
longitud_maxima = max(len(sublista) for sublista in HJD_periodicas + error_periodicas + Series_LPV + HJD_LPV + error_LPV)


# Llenar las listas con NaNs para igualar la longitud
Series_periodicas = llenar_con_nan(Series_periodicas, longitud_maxima)
HJD_periodicas = llenar_con_nan(HJD_periodicas, longitud_maxima)
error_periodicas = llenar_con_nan(error_periodicas, longitud_maxima)
Series_LPV = llenar_con_nan(Series_LPV, longitud_maxima)
HJD_LPV = llenar_con_nan(HJD_LPV, longitud_maxima)
error_LPV = llenar_con_nan(error_LPV, longitud_maxima)

# Convertir las listas de listas a arrays de NumPy
Series_periodicas_array = np.array(Series_periodicas)
HJD_periodicas_array = np.array(HJD_periodicas)
error_periodicas_array = np.array(error_periodicas)
Series_LPV_array = np.array(Series_LPV)
HJD_LPV_array = np.array(HJD_LPV)
error_LPV_array = np.array(error_LPV)

# Crear un HDU primario con un header vacío
primary_hdu = fits.PrimaryHDU()

# Crear objetos ImageHDU para cada array
hdu1 = fits.ImageHDU(Series_periodicas_array, name='S1_period')
hdu2 = fits.ImageHDU(HJD_periodicas_array, name='HJD1_period')
hdu3 = fits.ImageHDU(error_periodicas_array, name='E1_period')
hdu4 = fits.ImageHDU(Series_LPV_array, name='S1_LPV')
hdu5 = fits.ImageHDU(HJD_LPV_array, name='HJD1_LPV')
hdu6 = fits.ImageHDU(error_LPV_array, name='E1_LPV')


# Crear un HDUList y agregar los HDUs correspondientes
hdul = fits.HDUList([primary_hdu, hdu1, hdu2, hdu3, hdu4, hdu5, hdu6])

# Guardar el HDUList en un archivo FITS
hdul.writeto('Variablesb279.fits', overwrite=True)

print("Archivo FITS creado con éxito.")



Archivo FITS creado con éxito.
